In [ ]:
import torch
import cv2
import threading
import numpy as np
from tkinter import Tk, messagebox, Label, Button, filedialog
from PIL import Image,ImageTk


In [ ]:
from ultralytics import yolov5

model= torch.hub.load('ultralytics/yolov5','yolov5')

In [ ]:
carnivores = ['lion','tiger','cheetah','leapord','hyena','fox','polar bear']

In [ ]:
root= Tk()
root.title("Animal Detection System")
root.geometry("800x600")

img_label= Label(root)
img_label.pack()


In [ ]:
def detect_animals_in_image(filepath):
    img= cv2.imread(filepath)
    results= model(img)
    detections=results.pandas().xyxy[0]

    carnivore_count=0

    for _,row in detections.iterrows():
        label= row['name']
        x1,y1,x2,y2= int(row['xmin']),int(row['ymin']),int(row['xmax']),int(row['ymax'])
        color = (0,0,255) if label.lower() in carnivores else color=(0,255,0)

        if label.lower() in carnivores:
            carnivore_count +=1
        cv2.rectangle(img,(x1,y1),(x2,y2),color,2)
        cv2.putText(img,label,(x1, y1 - 10),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)

    messagebox.showinfo("Detection Result", f"Carnivores Detected: {carnivore_count}")

    img_rgb= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img_pil=Image.fromarray(img_rgb)
    img_tk=ImageTk.PhotoImage(image=img_pil)
    img_label.configure(image=img_tk)
    img_label.image= img_tk

def detect_animals_in_video(filepath):
    cap = cv2.VideoCapture(filepath)

    def process_video():
        carnivore_count=0
        while True:
            ret,frame=cap.read()
            if not ret:
                break

            results= model(frame)
            detections=results.pandas().xyxy[0]
            frame_carnivores = 0


            for _,row in detections.iterrows():
                label=row['name']
                x1,y1,x2,y2=int(row['xmin']),int(row['ymin']),int(row['xmax']),int(row['ymax'])
                color=(0,0,255) if label.lower() in carnivores else color=(0,255,0)
                if label.lower() in carnivores:
                    frame_carnivores +=1
                cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
                cv2.putText(frame,label, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            carnivore_count += frame_carnivores

            img_rgb= cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            img_pil=Image.fromarray(img_rgb)
            img_tk=ImageTk.PhotoImage(image=img_pil)
            img_label.configure(image=img_tk)
            img_label.image= img_tk
            root.update()

        cap.release()
        messagebox.showinfo("Video Analysis Completed",f"Total carnivores detected: {carnivore_count}")

    threading.Thread(target=process_video).start()

def browse_img():
    filepath=filedialog.askopenfilename(filetypes=[("Image files","*.jpg;*.jpeg;*.png")])
    if filepath:
        detect_animals_in_image(filepath)

def browse_video():
    filepath=filedialog.askopenfilename(filetypes=["Video files","*.mp4;*.avi;*.mov"])
    if filepath:
        detect_animals_in_video(filepath)

Button(root, text="Upload Image", command=browse_img,width=20, bg="skyblue").pack(pady=10)
Button(root, text="Upload Video", command= browse_video, width=20, bg="lightgreen").pack(pady=10)


root.mainloop()